# Libraries

In [24]:
import json
import requests
import pandas as pd
from pandas.io.json import json_normalize

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

import nltk.data
import spacy

import warnings
warnings.filterwarnings('ignore')

import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ferencipeter/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [25]:
from IPython.display import display, HTML

display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

# Import Data

In [36]:
data = pd.read_csv('data/fox_trump.csv', sep=";", encoding='latin-1')

In [37]:
data.head()

,Date,Candidate,Headline,Snippet
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...


In [38]:
data.shape

(205, 4)

In [39]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 4 columns):
Date         205 non-null object
Candidate    205 non-null object
Headline     205 non-null object
Snippet      203 non-null object
dtypes: object(4)
memory usage: 6.5+ KB


# Text Cleaning

In [40]:
def text_cleanup(text):
    text = text.replace('RT ', '')
    text = text.replace('\n', '')
    clean_text = []
    for word in text.split(' '):
        if word.startswith('http://'):
            word = ''
        if word.startswith('https://'):
            word = ''
        if word.startswith('.@'):
            word = word[1:]
        if word.endswith('.'):
            word = word[:-1]
        clean_text.append(word)
    return ' '.join(clean_text)


for i in data.index.tolist():
    data.at[i,'Headline_Cleaned'] = text_cleanup(data['Headline'][i])

In [41]:
for i in data.index.tolist():
    data.at[i,'Snippet_Cleaned'] = text_cleanup(data['Snippet'][i])

AttributeError: 'float' object has no attribute 'replace'

In [42]:
data.head()

,Date,Candidate,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...


# Stop Words and Tokenization

In [43]:
nlp = spacy.load('en_core_web_sm')
from spacy.lang.en.stop_words import STOP_WORDS

data['Headline_Keywords'] = ""
data['Headline_Stopwords'] = ""
data['Headline_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Headline_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Headline_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Headline_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

---------
John Sununu reverses position, endorses Trump
---------
---------
['John', 'Sununu', 'reverses', 'position', ',', 'endorses', 'Trump']
[]
---------
---------
Campaign adviser: Trump wins the debate over the economy
---------
---------
['Campaign', 'adviser', ':', 'Trump', 'wins', 'debate', 'economy']
['the', 'over', 'the']
---------
---------
Women who know Trump defend the GOP candidate's character
---------
---------
['Women', 'know', 'Trump', 'defend', 'GOP', 'candidate', 'character']
['who', 'the', "'s"]
---------
---------
Trump's economic agenda: Cutting taxes for job creation
---------
---------
['Trump', 'economic', 'agenda', ':', 'Cutting', 'taxes', 'job', 'creation']
["'s", 'for']
---------
---------
Presidential campaign spiraling downward with talk of sex tapes
---------
---------
['Presidential', 'campaign', 'spiraling', 'downward', 'talk', 'sex', 'tapes']
['with', 'of']
---------
---------
Krauthammer: Trump needs to stop falling for traps
---------
---------
['

---------
Is there still time for a Trump victory?
---------
---------
['time', 'Trump', 'victory', '?']
['Is', 'there', 'still', 'for', 'a']
---------
---------
Fox Flash: Mini-Trump
---------
---------
['Fox', 'Flash', ':', 'Mini', '-', 'Trump']
[]
---------
---------
Crowd goes wild over 'mini-Trump'
---------
---------
['Crowd', 'goes', 'wild', "'", 'mini', '-', 'Trump', "'"]
['over']
---------
---------
Donald Trump on taking 'shackles' off campaign
---------
---------
['Donald', 'Trump', 'taking', "'", 'shackles', "'", 'campaign']
['on', 'off']
---------
---------
Timing of the Trump video, accusers suspicious?
---------
---------
['Timing', 'Trump', 'video', ',', 'accusers', 'suspicious', '?']
['of', 'the']
---------
---------
Trump fights back after NY Times attack
---------
---------
['Trump', 'fights', 'NY', 'Times', 'attack']
['back', 'after']
---------
---------
Multiple women accuse Donald Trump of groping
---------
---------
['Multiple', 'women', 'accuse', 'Donald', 'Trum

---------
Trump announces plan to visit 'Democrat strongholds'
---------
---------
['Trump', 'announces', 'plan', 'visit', "'", 'Democrat', 'strongholds', "'"]
['to']
---------
---------
The forgotten women who are supporting Trump
---------
---------
['forgotten', 'women', 'supporting', 'Trump']
['The', 'who', 'are']
---------
---------
Clinton and Trump make final push to win battleground states
---------
---------
['Clinton', 'Trump', 'final', 'push', 'win', 'battleground', 'states']
['and', 'make', 'to']
---------
---------
Lara Trump: Women are so excited to vote Trump
---------
---------
['Lara', 'Trump', ':', 'Women', 'excited', 'vote', 'Trump']
['are', 'so', 'to']
---------
---------
Pundits say Trump could win
---------
---------
['Pundits', 'Trump', 'win']
['say', 'could']
---------
---------
Donald Trump shows his softer side
---------
---------
['Donald', 'Trump', 'shows', 'softer']
['his', 'side']
---------
---------
Sarah Palin: If Trump wins, America wins
---------
-----

---------
Trump chooses ObamaCare critic to revamp health care system
---------
---------
['Trump', 'chooses', 'ObamaCare', 'critic', 'revamp', 'health', 'care', 'system']
['to']
---------
---------
Jarrett: Clinton did the one thing guaranteed to rile Trump
---------
---------
['Jarrett', ':', 'Clinton', 'thing', 'guaranteed', 'rile', 'Trump']
['did', 'the', 'one', 'to']
---------
---------
Kurtz: Can Trump appointees drain the swamp?
---------
---------
['Kurtz', ':', 'Trump', 'appointees', 'drain', 'swamp', '?']
['Can', 'the']
---------
---------
Palestinian leader seeks Trump support for independence
---------
---------
['Palestinian', 'leader', 'seeks', 'Trump', 'support', 'independence']
['for']
---------
---------
Trump formally announces Ross for Commerce secretary
---------
---------
['Trump', 'formally', 'announces', 'Ross', 'Commerce', 'secretary']
['for']
---------


In [11]:
data['Snippet_Keywords'] = ""
data['Snippet_Stopwords'] = ""
data['Snippet_Tokens'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet_Cleaned'][i])
    
    print('---------')
    print(doc)
    print('---------')
        
    token_list = []
    for token in doc:
        token_list.append(token.text)
    # Create list of word tokens after removing stopwords
    filtered_sentence =[] 
    stopwords =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
        else:
            stopwords.append(word)
            
    data.at[i,'Snippet_Keywords'] = ', '.join(filtered_sentence)
    data.at[i,'Snippet_Stopwords'] = ', '.join(stopwords)    
    data.at[i,'Snippet_Tokens'] = ', '.join(token_list)    
        
    print('---------')
    print(filtered_sentence)
    print(stopwords)
    print('---------')

---------
A case for Trump’s ideas that ignores the man himself
---------
---------
['case', 'Trump', 'ideas', 'ignores', 'man']
['A', 'for', '’s', 'that', 'the', 'himself']
---------
---------
Unlike her prominent half siblings, Tiffany Trump has rarely been seen on the presidential campaign trail in 2016 But that may be about to change
---------
---------
['Unlike', 'prominent', 'half', 'siblings', ',', 'Tiffany', 'Trump', 'rarely', 'seen', 'presidential', 'campaign', 'trail', '2016', 'change']
['her', 'has', 'been', 'on', 'the', 'in', 'But', 'that', 'may', 'be', 'about', 'to']
---------
---------
The Republican nominee has proposed, in effect, to cut spending on infrastructure as part of his plan to borrow for spending on infrastructure
---------
---------
['Republican', 'nominee', 'proposed', ',', 'effect', ',', 'cut', 'spending', 'infrastructure', 'plan', 'borrow', 'spending', 'infrastructure']
['The', 'has', 'in', 'to', 'on', 'as', 'part', 'of', 'his', 'to', 'for', 'on']
--------

---------
Donald J Trump said at a campaign rally in Pueblo, Colo., that he had used the country's tax laws to his benefit “as legally required” and hated the way the current administration spends tax dollars
---------
---------
['Donald', 'J', 'Trump', 'said', 'campaign', 'rally', 'Pueblo', ',', 'Colo.', ',', 'country', 'tax', 'laws', 'benefit', '“', 'legally', 'required', '”', 'hated', 'way', 'current', 'administration', 'spends', 'tax', 'dollars']
['at', 'a', 'in', 'that', 'he', 'had', 'used', 'the', "'s", 'to', 'his', 'as', 'and', 'the', 'the']
---------
---------
Hillary Clinton, responding to the New York Times article on Donald J Trump’s 1995 tax return, told a rally in Toledo, Ohio, that there should be a law requiring major party nominees to release their taxes
---------
---------
['Hillary', 'Clinton', ',', 'responding', 'New', 'York', 'Times', 'article', 'Donald', 'J', 'Trump', '1995', 'tax', 'return', ',', 'told', 'rally', 'Toledo', ',', 'Ohio', ',', 'law', 'requiring', 'ma

---------
Why Larry Summers, Robert Reich and other economists think the Republican presidential candidate’s trade policies are ‘lunatic.’
---------
---------
['Larry', 'Summers', ',', 'Robert', 'Reich', 'economists', 'think', 'Republican', 'presidential', 'candidate', 'trade', 'policies', '‘', 'lunatic', '.', '’']
['Why', 'and', 'other', 'the', '’s', 'are']
---------
---------
Donald J Trump is a prolific, if controversial, businessman Here, we examine some of his ventures to find out how they fared (Pictured: Mr Trump showing off his home furnishings collection in 2007.)
---------
---------
['Donald', 'J', 'Trump', 'prolific', ',', 'controversial', ',', 'businessman', ',', 'examine', 'ventures', 'find', 'fared', '(', 'Pictured', ':', 'Mr', 'Trump', 'showing', 'home', 'furnishings', 'collection', '2007', '.', ')']
['is', 'a', 'if', 'Here', 'we', 'some', 'of', 'his', 'to', 'out', 'how', 'they', 'off', 'his', 'in']
---------
---------
Of roughly 60 ventures started or promoted by Mr Tru

---------
Top NBC officials learned early last week of the video of Donald J Trump making vulgar remarks about women But it was The Washington Post that broke the story
---------
---------
['NBC', 'officials', 'learned', 'early', 'week', 'video', 'Donald', 'J', 'Trump', 'making', 'vulgar', 'remarks', 'women', 'Washington', 'Post', 'broke', 'story']
['Top', 'last', 'of', 'the', 'of', 'about', 'But', 'it', 'was', 'The', 'that', 'the']
---------
---------
Mr Trump called the defectors “self-righteous hypocrites” and predicted their defeat at the ballot box
---------
---------
['Mr', 'Trump', 'called', 'defectors', '“', 'self', '-', 'righteous', 'hypocrites', '”', 'predicted', 'defeat', 'ballot', 'box']
['the', 'and', 'their', 'at', 'the']
---------
---------
Men of many backgrounds and parts of the country had varied opinions on how men talk, but they agreed that Mr Trump’s version was unacceptable
---------
---------
['Men', 'backgrounds', 'parts', 'country', 'varied', 'opinions', 'men',

---------
---------
Has he gone too far, at last?
---------
---------
['gone', 'far', ',', '?']
['Has', 'he', 'too', 'at', 'last']
---------
---------
A survivor of rape reflects on the Trump tape
---------
---------
['survivor', 'rape', 'reflects', 'Trump', 'tape']
['A', 'of', 'on', 'the']
---------
---------
On Twitter, Mr Trump criticized Republicans who had withdrawn support after a recording of him demeaning women became public
---------
---------
['Twitter', ',', 'Mr', 'Trump', 'criticized', 'Republicans', 'withdrawn', 'support', 'recording', 'demeaning', 'women', 'public']
['On', 'who', 'had', 'after', 'a', 'of', 'him', 'became']
---------
---------
Recordings of his vulgar boasts would make the Republican nominee a major legal liability, and many companies consider such language a firing offense
---------
---------
['Recordings', 'vulgar', 'boasts', 'Republican', 'nominee', 'major', 'legal', 'liability', ',', 'companies', 'consider', 'language', 'firing', 'offense']
['of', 'his

...

---------
Women’s loyalty to class and party trump their loyalty to one another
---------
---------
['Women', 'loyalty', 'class', 'party', 'trump', 'loyalty']
['’s', 'to', 'and', 'their', 'to', 'one', 'another']
---------
---------
Party has long outweighed gender when people vote This election turned out to be no different
---------
---------
['Party', 'long', 'outweighed', 'gender', 'people', 'vote', 'election', 'turned', 'different']
['has', 'when', 'This', 'out', 'to', 'be', 'no']
---------
---------
The seeds of the biggest upset in American politics in recent memory were sewn in the Midwest, where decades of economic decay largely ignored by Democrats came back to haunt Hillary Clinton
---------
---------
['seeds', 'biggest', 'upset', 'American', 'politics', 'recent', 'memory', 'sewn', 'Midwest', ',', 'decades', 'economic', 'decay', 'largely', 'ignored', 'Democrats', 'came', 'haunt', 'Hillary', 'Clinton']
['The', 'of', 'the', 'in', 'in', 'were', 'in', 'the', 'where', 'of', 'by', 

---------
After losing the presidency, Congress and lower-level races, the party is divided over how aggressively to position itself on the economic left
---------
---------
['losing', 'presidency', ',', 'Congress', 'lower', '-', 'level', 'races', ',', 'party', 'divided', 'aggressively', 'position', 'economic', 'left']
['After', 'the', 'and', 'the', 'is', 'over', 'how', 'to', 'itself', 'on', 'the']
---------
---------
The leaders discussed trade and combating terrorism, with the Russian president saying he hoped for “noninterference in the other’s internal affairs.”
---------
---------
['leaders', 'discussed', 'trade', 'combating', 'terrorism', ',', 'Russian', 'president', 'saying', 'hoped', '“', 'noninterference', 'internal', 'affairs', '.', '”']
['The', 'and', 'with', 'the', 'he', 'for', 'in', 'the', 'other', '’s']
---------
---------
Democrats need to borrow his tactics
---------
---------
['Democrats', 'need', 'borrow', 'tactics']
['to', 'his']
---------
---------
Long waits at pol

---------
We keep the powerful accountable — and that will be even more important in the coming years
---------
---------
['powerful', 'accountable', '—', 'important', 'coming', 'years']
['We', 'keep', 'the', 'and', 'that', 'will', 'be', 'even', 'more', 'in', 'the']
---------
---------
A reader points out that Democrats won the popular vote for president and made gains in the Senate and the House
---------
---------
['reader', 'points', 'Democrats', 'won', 'popular', 'vote', 'president', 'gains', 'Senate', 'House']
['A', 'out', 'that', 'the', 'for', 'and', 'made', 'in', 'the', 'and', 'the']
---------
---------
Here’s what you need to know to start your day
---------
---------
['need', 'know', 'start', 'day']
['Here', '’s', 'what', 'you', 'to', 'to', 'your']
---------
---------
How is it possible that the supposedly fading constituency of white working class voters played such a decisive role in the 2016 election?
---------
---------
['possible', 'supposedly', 'fading', 'constituency', 

---------
Michelle Obama and noted journalists attended a memorial service at Metropolitan A.M.E Church for Ms Ifill, the PBS anchor, who died of cancer on Monday
---------
---------
['Michelle', 'Obama', 'noted', 'journalists', 'attended', 'memorial', 'service', 'Metropolitan', 'A.M.E', 'Church', 'Ms', 'Ifill', ',', 'PBS', 'anchor', ',', 'died', 'cancer', 'Monday']
['and', 'a', 'at', 'for', 'the', 'who', 'of', 'on']
---------
---------
As Democrats pick through the wreckage of the campaign, one lesson is clear: The election was notable as much for the people who did not show up, as for those who did
---------
---------
['Democrats', 'pick', 'wreckage', 'campaign', ',', 'lesson', 'clear', ':', 'election', 'notable', 'people', ',']
['As', 'through', 'the', 'of', 'the', 'one', 'is', 'The', 'was', 'as', 'much', 'for', 'the', 'who', 'did', 'not', 'show', 'up', 'as', 'for', 'those', 'who', 'did']
---------
---------
The stream of job candidates visiting the president-elect continued as tran

---------
A reader is troubled by Trump’s presumption that whether to prosecute or not is his decision
---------
---------
['reader', 'troubled', 'Trump', 'presumption', 'prosecute', 'decision']
['A', 'is', 'by', '’s', 'that', 'whether', 'to', 'or', 'not', 'is', 'his']
---------
---------
President-elect Donald J Trump has chosen Gov Nikki Haley of South Carolina to be ambassador to the United Nations
---------
---------
['President', '-', 'elect', 'Donald', 'J', 'Trump', 'chosen', 'Gov', 'Nikki', 'Haley', 'South', 'Carolina', 'ambassador', 'United', 'Nations']
['has', 'of', 'to', 'be', 'to', 'the']
---------
---------
His meeting with The Times was a marvel of boasting and bending
---------
---------
['meeting', 'Times', 'marvel', 'boasting', 'bending']
['His', 'with', 'The', 'was', 'a', 'of', 'and']
---------
---------
Representative Keith Ellison of Minnesota has been steadily adding endorsements, but some advisers to President Obama are urging others to enter the race
---------
---

In [44]:
data.head()

,Date,Candidate,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John, Sununu, reverses, position, ,, endorses,...",,"John, Sununu, reverses, position, ,, endorses,..."
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,"Campaign, adviser, :, Trump, wins, debate, eco...","the, over, the","Campaign, adviser, :, Trump, wins, the, debate..."
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,"Women, know, Trump, defend, GOP, candidate, ch...","who, the, 's","Women, who, know, Trump, defend, the, GOP, can..."
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,"Trump, economic, agenda, :, Cutting, taxes, jo...","'s, for","Trump, 's, economic, agenda, :, Cutting, taxes..."
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,"Presidential, campaign, spiraling, downward, t...","with, of","Presidential, campaign, spiraling, downward, w..."


# Sentiment

#### "VADER has been found to be quite successful when dealing with social media texts, NY Times editorials, movie reviews, and product reviews. "

With VADER no need for removing stop words as its built-in and punctuation is part of the analysis

In [45]:
import spacy
from spacy.tokens import Doc
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nlp = spacy.load('en_core_web_sm') 
sentiment_analyzer = SentimentIntensityAnalyzer()
def polarity_scores(doc):
    return sentiment_analyzer.polarity_scores(doc.text)
 
Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Sentiment_Values'] = ""
data['Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Headline'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Sentiment_Values'] = senti   
    data.at[i,'Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

---------
John Sununu reverses position, endorses Trump
---------
---------
[{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'compound': 0.34}]
---------
---------
Campaign adviser: Trump wins the debate over the economy
---------
---------
[{'neg': 0.0, 'neu': 0.684, 'pos': 0.316, 'compound': 0.5719}]
---------
---------
Women who know Trump defend the GOP candidate's character
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump's economic agenda: Cutting taxes for job creation
---------
---------
[{'neg': 0.156, 'neu': 0.625, 'pos': 0.219, 'compound': 0.1531}]
---------
---------
Presidential campaign spiraling downward with talk of sex tapes
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Krauthammer: Trump needs to stop falling for traps
---------
---------
[{'neg': 0.388, 'neu': 0.612, 'pos': 0.0, 'compound': -0.4215}]
---------
---------
Ben Carson reacts to Trump's attacks on former Miss U

---------
Trump blames Mexican billionaire for harassment stories
---------
---------
[{'neg': 0.554, 'neu': 0.446, 'pos': 0.0, 'compound': -0.7351}]
---------
---------
Trump blames groping claims on media, Clinton
---------
---------
[{'neg': 0.31, 'neu': 0.69, 'pos': 0.0, 'compound': -0.4019}]
---------
---------
Donald Trump says allegations are 100% fabricated
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Never Trump' Republican: Voters are increasingly disgusted
---------
---------
[{'neg': 0.362, 'neu': 0.638, 'pos': 0.0, 'compound': -0.5267}]
---------
---------
Trump campaign reports raising $2 million in September
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump spokeswoman rips media
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Miss USA 2005 defends Trump: He has always been respectful
---------
---------
[{'neg': 0.127, 'n

---------
Clinton and Trump make final push to win battleground states
---------
---------
[{'neg': 0.186, 'neu': 0.552, 'pos': 0.262, 'compound': 0.2732}]
---------
---------
Lara Trump: Women are so excited to vote Trump
---------
---------
[{'neg': 0.0, 'neu': 0.72, 'pos': 0.28, 'compound': 0.4795}]
---------
---------
Pundits say Trump could win
---------
---------
[{'neg': 0.0, 'neu': 0.513, 'pos': 0.487, 'compound': 0.5859}]
---------
---------
Donald Trump shows his softer side
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Sarah Palin: If Trump wins, America wins
---------
---------
[{'neg': 0.0, 'neu': 0.403, 'pos': 0.597, 'compound': 0.8126}]
---------
---------
Why voters trust Trump with money issues
---------
---------
[{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'compound': 0.5106}]
---------
---------
Krauthammer: Donald Trump will be the power broker
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]

---------
President-elect Trump building foreign policy team
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump nominees could face resistance in Congress
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump takes to Twitter to bypass, criticize the medi
---------
---------
[{'neg': 0.245, 'neu': 0.755, 'pos': 0.0, 'compound': -0.3818}]
---------
---------
Is Trump abandoning promises?
---------
---------
[{'neg': 0.361, 'neu': 0.278, 'pos': 0.361, 'compound': 0.0}]
---------
---------
President-elect Trump works to build administration
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
President-elect Trump calls recount efforts a 'scam'
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump still facing concerns about white supremacists
---------
---------
[{'neg': 0.25, 'neu': 0.75, 'pos': 0.0,

In [46]:
#most headlines seem to fall under neutral, maybe better to paint directly with the polarity metric

data.head()

,Date,Candidate,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John, Sununu, reverses, position, ,, endorses,...",,"John, Sununu, reverses, position, ,, endorses,...","[{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'com...",neu
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,"Campaign, adviser, :, Trump, wins, debate, eco...","the, over, the","Campaign, adviser, :, Trump, wins, the, debate...","[{'neg': 0.0, 'neu': 0.684, 'pos': 0.316, 'com...",neu
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,"Women, know, Trump, defend, GOP, candidate, ch...","who, the, 's","Women, who, know, Trump, defend, the, GOP, can...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,"Trump, economic, agenda, :, Cutting, taxes, jo...","'s, for","Trump, 's, economic, agenda, :, Cutting, taxes...","[{'neg': 0.156, 'neu': 0.625, 'pos': 0.219, 'c...",neu
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,"Presidential, campaign, spiraling, downward, t...","with, of","Presidential, campaign, spiraling, downward, w...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu


In [47]:
#let's try analysing the snippets as well

Doc.set_extension('polarity_scores', getter=polarity_scores,force=True)

data['Snippet_Sentiment_Values'] = ""
data['Snippet_Sentiment'] = ""

for i in data.index.tolist():
    doc = nlp(data['Snippet'][i])
    
    print('---------')
    print(doc)
    print('---------')
                
    senti = []
    senti.append(doc._.polarity_scores)
        
    data.at[i,'Snippet_Sentiment_Values'] = senti   
    data.at[i,'Snippet_Sentiment'] = sorted(senti[0], key=senti[0].__getitem__)[-1]
    
    print('---------')
    print(senti)
    print('---------')

---------
Former New Hampshire governor explains his decision to support the GOP candidate
---------
---------
[{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'compound': 0.4019}]
---------
---------
Fox News contributor and economic adviser to Trump explains on 'America's Election HQ' what issues he hopes the candidate will address at future debates
---------
---------
[{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'compound': 0.4215}]
---------
---------
Former VP of Trump International and former pageant contestants speak out on 'Hannity'
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Trump senior economic adviser Steve Moore explains the strategy
---------
---------
[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}]
---------
---------
Donald Trump is encouraging voters to check out a "sex tape" featuring the former beauty queen with whom he's feuding. Hillary Clinton's campaign is suggesting that a better rental is the adult film in which

TypeError: object of type 'float' has no len()

In [48]:
data.head()

,Date,Candidate,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John, Sununu, reverses, position, ,, endorses,...",,"John, Sununu, reverses, position, ,, endorses,...","[{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'com...",neu,"[{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'com...",neu
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,"Campaign, adviser, :, Trump, wins, debate, eco...","the, over, the","Campaign, adviser, :, Trump, wins, the, debate...","[{'neg': 0.0, 'neu': 0.684, 'pos': 0.316, 'com...",neu,"[{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'com...",neu
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,"Women, know, Trump, defend, GOP, candidate, ch...","who, the, 's","Women, who, know, Trump, defend, the, GOP, can...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,"Trump, economic, agenda, :, Cutting, taxes, jo...","'s, for","Trump, 's, economic, agenda, :, Cutting, taxes...","[{'neg': 0.156, 'neu': 0.625, 'pos': 0.219, 'c...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,"Presidential, campaign, spiraling, downward, t...","with, of","Presidential, campaign, spiraling, downward, w...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.058, 'neu': 0.716, 'pos': 0.226, 'c...",compound


In [49]:
data['Sentiment_Values'][2][0]['compound']

0.0

In [50]:
data.Headline.iloc[0]

'John Sununu reverses position, endorses Trump'

In [51]:
data['Headline_polarity'] = data['Headline']

for i in range(len(data.Sentiment)):
    data['Headline_polarity'].iloc[i] = data['Sentiment_Values'][i][0]['compound']

In [52]:
data['Snippet_polarity'] = data['Headline']

for i in range(len(data.Sentiment)):
    data['Snippet_polarity'].iloc[i] = data['Snippet_Sentiment_Values'][i][0]['compound']

IndexError: string index out of range

In [53]:
data.head(30)

,Date,Candidate,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment,Headline_polarity,Snippet_polarity
0,01/10/2016,Trump,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John Sununu reverses position, endorses Trump",Former New Hampshire governor explains his dec...,"John, Sununu, reverses, position, ,, endorses,...",,"John, Sununu, reverses, position, ,, endorses,...","[{'neg': 0.0, 'neu': 0.676, 'pos': 0.324, 'com...",neu,"[{'neg': 0.0, 'neu': 0.803, 'pos': 0.197, 'com...",neu,0.34,0.4019
1,01/10/2016,Trump,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,Campaign adviser: Trump wins the debate over t...,Fox News contributor and economic adviser to T...,"Campaign, adviser, :, Trump, wins, debate, eco...","the, over, the","Campaign, adviser, :, Trump, wins, the, debate...","[{'neg': 0.0, 'neu': 0.684, 'pos': 0.316, 'com...",neu,"[{'neg': 0.0, 'neu': 0.891, 'pos': 0.109, 'com...",neu,0.5719,0.4215
2,01/10/2016,Trump,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,Women who know Trump defend the GOP candidate'...,Former VP of Trump International and former pa...,"Women, know, Trump, defend, GOP, candidate, ch...","who, the, 's","Women, who, know, Trump, defend, the, GOP, can...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0
3,01/10/2016,Trump,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,Trump's economic agenda: Cutting taxes for job...,Trump senior economic adviser Steve Moore expl...,"Trump, economic, agenda, :, Cutting, taxes, jo...","'s, for","Trump, 's, economic, agenda, :, Cutting, taxes...","[{'neg': 0.156, 'neu': 0.625, 'pos': 0.219, 'c...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0.1531,0
4,01/10/2016,Trump,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,Presidential campaign spiraling downward with ...,Donald Trump is encouraging voters to check ou...,"Presidential, campaign, spiraling, downward, t...","with, of","Presidential, campaign, spiraling, downward, w...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.058, 'neu': 0.716, 'pos': 0.226, 'c...",compound,0,0.8176
5,01/10/2016,Trump,Krauthammer: Trump needs to stop falling for t...,Syndicated columnist offers advice to both of ...,Krauthammer: Trump needs to stop falling for t...,Syndicated columnist offers advice to both of ...,"Krauthammer, :, Trump, needs, stop, falling, t...","to, for","Krauthammer, :, Trump, needs, to, stop, fallin...","[{'neg': 0.388, 'neu': 0.612, 'pos': 0.0, 'com...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,-0.4215,0
6,01/10/2016,Trump,Ben Carson reacts to Trump's attacks on former...,Trump supporter hopes the GOP candidate will f...,Ben Carson reacts to Trump's attacks on former...,Trump supporter hopes the GOP candidate will f...,"Ben, Carson, reacts, Trump, attacks, Miss, Uni...","to, 's, on, former","Ben, Carson, reacts, to, Trump, 's, attacks, o...","[{'neg': 0.36, 'neu': 0.64, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 0.647, 'pos': 0.353, 'com...",neu,-0.5423,0.5994
7,01/10/2016,Trump,Trump looks to get past tweet frenzy with stop...,Donald Trump is on the campaign trail Saturday...,Trump looks to get past tweet frenzy with stop...,Donald Trump is on the campaign trail Saturday...,"Trump, looks, past, tweet, frenzy, stop, swing...","to, get, with, in","Trump, looks, to, get, past, tweet, frenzy, wi...","[{'neg': 0.29, 'neu': 0.71, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.117, 'neu': 0.741, 'pos': 0.141, 'c...",neu,-0.5423,0.34
8,01/10/2016,Trump,Kurtz on Trump's Miss Universe dust up: 'He ca...,Howard Kurtz told viewers Friday on

# Frequency

In [22]:
#for frquency need to tokenize
#to tokenize need to clear punctuations

from collections import Counter

def wordextractdf(df, col):
    """
    Extract words in the dataframe

    Inputs:
    df -- dataframe
    col -- set of words that we can extract ('keywords', 'hashtags',...)

    Outputs:
    wordsfreq - Counter containing a list of words and its frequency in the dataframe
    """
    words = df[col][df[col].isnull() == False]
    lst = [w for word in words for w in word.split(",")]
    return Counter(lst)

def calc_freq(df, col):
    kw = wordextractdf(df, col)
    dictlist = [[key, value] for key, value in kw.items()]
    freq_df = pd.DataFrame()
    freq_df['words'] = [i[0] for i in dictlist]
    freq_df['freq_total'] = [i[1] for i in dictlist]
    return freq_df

sorted(wordextractdf(data, 'Headline_Keywords').most_common())
calc_freq(data, 'Headline_Keywords')

,words,freq_total
0,Trump,144
1,Intellectuals,1
2,Infrastructure,2
3,Plan,7
4,Fatal,2
5,Flaw,2
6,Donald,173
7,Trump,578
8,Pathetic,1
9,Fraternity,2


# Export

In [54]:
data.to_csv('data/fox_trump_sentiment.csv', index=False)

In [104]:
data

,Candidate,Date,Headline,Snippet,Headline_Cleaned,Snippet_Cleaned,Headline_Keywords,Headline_Stopwords,Headline_Tokens,Snippet_Keywords,Snippet_Stopwords,Snippet_Tokens,Sentiment_Values,Sentiment,Snippet_Sentiment_Values,Snippet_Sentiment,Headline_polarity,Snippet_polarity
0,trump,2016-10-01,Trump and the Intellectuals,A case for Trump’s ideas that ignores the man ...,Trump and the Intellectuals,A case for Trump’s ideas that ignores the man ...,"Trump, Intellectuals","and, the","Trump, and, the, Intellectuals","case, Trump, ideas, ignores, man","A, for, ’s, that, the, himself","A, case, for, Trump, ’s, ideas, that, ignores,...","[{'neg': 0.0, 'neu': 0.536, 'pos': 0.464, 'com...",neu,"[{'neg': 0.208, 'neu': 0.792, 'pos': 0.0, 'com...",neu,0.3818,-0.2732
1,trump,2016-10-01,The Other Trump,"Unlike her prominent half siblings, Tiffany Tr...",The Other Trump,"Unlike her prominent half siblings, Tiffany Tr...",Trump,"The, Other","The, Other, Trump","Unlike, prominent, half, siblings, ,, Tiffany,...","her, has, been, on, the, in, But, that, may, b...","Unlike, her, prominent, half, siblings, ,, Tif...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 0.913, 'pos': 0.087, 'com...",neu,0,0.3182
2,trump,2016-10-01,Donald Trump’s Pathetic Fraternity,Chris Christie and Rudy Giuliani once had prid...,Donald Trump’s Pathetic Fraternity,Chris Christie and Rudy Giuliani once had prid...,"Donald, Trump, Pathetic, Fraternity",’s,"Donald, Trump, ’s, Pathetic, Fraternity","Chris, Christie, Rudy, Giuliani, pride, orange...","and, once, had, Now, they, have, an","Chris, Christie, and, Rudy, Giuliani, once, ha...","[{'neg': 0.552, 'neu': 0.448, 'pos': 0.0, 'com...",neg,"[{'neg': 0.0, 'neu': 0.844, 'pos': 0.156, 'com...",neu,-0.5719,0.34
3,trump,2016-10-01,Trump Infrastructure Plan’s Fatal Flaw,"The Republican nominee has proposed, in effect...",Trump Infrastructure Plan’s Fatal Flaw,"The Republican nominee has proposed, in effect...","Trump, Infrastructure, Plan, Fatal, Flaw",’s,"Trump, Infrastructure, Plan, ’s, Fatal, Flaw","Republican, nominee, proposed, ,, effect, ,, c...","The, has, in, to, on, as, part, of, his, to, f...","The, Republican, nominee, has, proposed, ,, in...","[{'neg': 0.467, 'neu': 0.533, 'pos': 0.0, 'com...",neu,"[{'neg': 0.087, 'neu': 0.913, 'pos': 0.0, 'com...",neu,-0.5423,-0.2732
4,trump,2016-10-01,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,How Could Anyone Vote for Trump?,It’s time for Republican leaders to exercise a...,"Vote, Trump, ?","How, Could, Anyone, for","How, Could, Anyone, Vote, for, Trump, ?","time, Republican, leaders, exercise, little, l...","It, ’s, for, to, a","It, ’s, time, for, Republican, leaders, to, ex...","[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,0,0
5,trump,2016-10-01,Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr. Trump said he was reconsi...",Donald Trump Opens New Line of Attack on Hilla...,"In an interview, Mr Trump said he was reconsid...","Donald, Trump, Opens, New, Line, Attack, Hilla...","of, on, Her","Donald, Trump, Opens, New, Line, of, Attack, o...","interview, ,, Mr, Trump, said, reconsidering, ...","In, an, he, was, whether, he, would, back, if,...","In, an, interview, ,, Mr, Trump, said, he, was...","[{'neg': 0.22, 'neu': 0.78, 'pos': 0.0, 'compo...",neu,"[{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compoun...",neu,-0.4767,0
6,trump,2016-10-01,Report That Donald Trump Did Business in Cuba ...,Revelations that Mr. Trump may have violated t...,Report That Donald Trump Did Business in Cuba ...,Revelations that Mr Trump may have violated th...,"Report, Donald, Trump, Business, Cuba, Ups, An...","That, Did, in, the, in","Report, That, Donald, Trump, Did, Business, in...","Revelations, Mr, Trump, violated, ban, America...","that, may, have, the, on, in, as, the, is, up,...","Revelations, that, Mr, Trump, may, have, viola...","[{'neg': 0.0, 